#predicting stock price in 10 minutes

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [3]:
nifty_500df = pd.read_csv('ind_nifty500list.csv')
nifty_500df.head()

,Company Name,Industry,Symbol,Series,ISIN Code
0,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,INE466L01038
1,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017
2,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022
3,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025
4,AIA Engineering Ltd.,Capital Goods,AIAENG,EQ,INE212H01026


In [4]:
data = yf.download(tickers="GAIL"+".NS", interval='1m', period='7d')
data['Price'] = (data['High'] + data['Low']) / 2

[*********************100%%**********************]  1 of 1 completed


In [5]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-03-05 09:15:00+05:30,192.500000,193.000000,192.050003,192.149994,192.149994,0
2024-03-05 09:16:00+05:30,192.100006,192.449997,191.600006,192.350006,192.350006,551267
2024-03-05 09:17:00+05:30,192.399994,193.149994,192.399994,192.949997,192.949997,498080
2024-03-05 09:18:00+05:30,193.100006,193.100006,192.600006,192.750000,192.750000,299710
2024-03-05 09:19:00+05:30,192.850006,193.100006,192.750000,193.100006,193.100006,202434


In [7]:
try:
  X_train = pickle.load(open('X_train.pkl', 'rb'))
  y_train = pickle.load(open('y_train.pkl', 'rb'))
except:
  window = 60
  X_train = np.empty((0, 2*window))  # 'window' is your defined window size, here 60
  y_train = np.empty(0)  # This makes y_train a 1D array to start with

  for symbol in nifty_500df['Symbol']:
      data = yf.download(tickers=symbol+".NS", interval='1m', period='7d')
      if data['Close'][-1] > 100:
          continue
      prices = np.array(data['Close'])
      window = 60
      X_train_symbol = np.array([prices[i:i+window] for i in range(len(prices)-window-10)])
      y_train_symbol = np.array([prices[i+window+10] for i in range(len(prices)-window-10)])

      volumes = np.array(data['Volume'])
      X_train_symbol = np.column_stack((X_train_symbol, [volumes[i:i+window] for i in range(len(volumes)-window-10)]))

      # pickle.dump(X_train_symbol, open("pickles/X_train_"+symbol+".pkl", "wb"))
      # pickle.dump(y_train_symbol, open("pickles/y_train_"+symbol+".pkl", "wb"))

      X_train = np.concatenate((X_train, X_train_symbol), axis=0)
      y_train = np.concatenate((y_train, y_train_symbol), axis=0)

X_train.shape, y_train.shape

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

((76396, 60), (76396,))

In [8]:
# Save the data
pickle.dump(X_train, open('X_train.pkl', 'wb'))
pickle.dump(y_train, open('y_train.pkl', 'wb'))

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [10]:
X_train.shape[1]

60

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Bidirectional(LSTM(units=30, return_sequences=True), input_shape=(X_train.shape[1], 1)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(units=30, return_sequences=False)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=1))  # Output layer

model.compile(optimizer='adam', loss='mean_squared_error')

In [21]:
epochs_batch_size = [(20, 512)]

for e,b in epochs_batch_size:
  model.fit(X_train, y_train, epochs=e, batch_size=b, validation_split=0.2)

Epoch 1/20
120/120 [==============================] - 2s 19ms/step - loss: 1.8726 - val_loss: 1.6344
Epoch 2/20
120/120 [==============================] - 2s 18ms/step - loss: 1.6571 - val_loss: 1.3289
Epoch 3/20
120/120 [==============================] - 2s 18ms/step - loss: 1.5651 - val_loss: 1.4045
Epoch 4/20
120/120 [==============================] - 3s 21ms/step - loss: 1.5571 - val_loss: 1.3159
Epoch 5/20
120/120 [==============================] - 2s 19ms/step - loss: 1.5424 - val_loss: 1.3155
Epoch 6/20
120/120 [==============================] - 2s 17ms/step - loss: 1.4902 - val_loss: 1.1064
Epoch 7/20
120/120 [==============================] - 2s 18ms/step - loss: 1.4718 - val_loss: 1.3173
Epoch 8/20
120/120 [==============================] - 2s 18ms/step - loss: 1.4582 - val_loss: 1.0550
Epoch 9/20
120/120 [==============================] - 2s 19ms/step - loss: 1.4336 - val_loss: 1.0971
Epoch 10/20
120/120 [==============================] - 2s 21ms/step - loss: 1.4265 - val_lo

In [22]:
model.save('predict_10m.h5')
from google.colab import files
files.download('predict_10m.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>